In [ ]:
# leer df_sin_anomalos.csv
import pandas as pd
df = pd.read_csv('df_sin_anomalos.csv')
df_final = pd.read_csv("df_final.csv")

In [ ]:
# dividir en train y test segun indice_test.csv y indice_train.csv
indice_test = pd.read_csv("indices_test.csv")
indice_train = pd.read_csv("indices_train.csv")

df_test = df_final.iloc[indice_test["0"],:]
df_train = df_final.iloc[indice_train["0"],:]

In [ ]:
X_test_todos = df_test.drop(['TRT'], axis=1)
y_test = pd.DataFrame(df_test["TRT"])

X_train_todos = df_train.drop(['TRT'], axis=1)
y_train = pd.DataFrame(df_train["TRT"])

In [ ]:
X_test_final = X_test_todos[["ARTERIOPATIA_PERIFERICA","ENFERMEDAD_VASCULAR","SAME_MAS_DE_2","N_PASTILLAS_D_INICIO","NEOPLASIA_NO","ADE","MONOCITOS","DISLIPEMIA","AMIODARONA","EDAD","INSUFICIENCIA_CARDIACA","SAME-TT2R2_MAS3","NEUTROFILOS"]]
X_train_final = X_train_todos[["ARTERIOPATIA_PERIFERICA","ENFERMEDAD_VASCULAR","SAME_MAS_DE_2","N_PASTILLAS_D_INICIO","NEOPLASIA_NO","ADE","MONOCITOS","DISLIPEMIA","AMIODARONA","EDAD","INSUFICIENCIA_CARDIACA","SAME-TT2R2_MAS3","NEUTROFILOS"]]

In [ ]:
y_train["TRT_60"] = 0
y_train.loc[y_train["TRT"] >= 60, "TRT_60"] = 1

y_test["TRT_60"] = 0
y_test.loc[y_test["TRT"] >= 60, "TRT_60"] = 1

y_train["TRT_65"] = 0
y_train.loc[y_train["TRT"] >= 65, "TRT_65"] = 1

y_test["TRT_65"] = 0
y_test.loc[y_test["TRT"] >= 65, "TRT_65"] = 1

y_train["TRT_70"] = 0
y_train.loc[y_train["TRT"] >= 70, "TRT_70"] = 1

y_test["TRT_70"] = 0
y_test.loc[y_test["TRT"] >= 70, "TRT_70"] = 1

In [ ]:
# undersampling + oversampling
from imblearn.over_sampling import ADASYN
sm = ADASYN(random_state=123)
X_train_todos60, y_train_bi60t = sm.fit_resample(X_train_todos, y_train["TRT_60"])
X_train_final60, y_train_bi60 = sm.fit_resample(X_train_final, y_train["TRT_60"])
X_train_todos65, y_train_bi65t = sm.fit_resample(X_train_todos, y_train["TRT_65"])
X_train_final65, y_train_bi65 = sm.fit_resample(X_train_final, y_train["TRT_65"])
X_train_todos70, y_train_bi70t = sm.fit_resample(X_train_todos, y_train["TRT_70"])
X_train_final70, y_train_bi70 = sm.fit_resample(X_train_final, y_train["TRT_70"])

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.calibration import calibration_curve

from sklearn.model_selection import GridSearchCV
from scipy.optimize import minimize
from sklearn.metrics import balanced_accuracy_score
# warnings
import warnings

# numpy
import numpy as np

# time
import time


# make scorer
from sklearn.metrics import make_scorer

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
# roc_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

def train_model_gridsearch2(model, X_train, X_val, y_train, y_val, params,DF):
    warnings.filterwarnings('ignore')
    grid = GridSearchCV(model, params, cv=5, scoring=scoring, refit="precision")
    grid.fit(X_train, y_train)
    print("Best params: ", grid.best_params_)
    print("Best score: ", grid.best_score_)
    start_time = time.time()
    y_pred = grid.predict(X_val)
    execution_time = (time.time() - start_time)
    print("F1: ", f1_score(y_val, y_pred))
    print("Precision: ", precision_score(y_val, y_pred))
    print("Recall: ", recall_score(y_val, y_pred))
    print("Balanced accuracy: ", accuracy_score(y_val, y_pred))
    # negative predictive value
    print("Negative predictive value: ", np.round(confusion_matrix(y_val, y_pred)[0][0]/(confusion_matrix(y_val, y_pred)[0][0]+confusion_matrix(y_val, y_pred)[1][0]), 2))
    # positive predictive value
    print("Positive predictive value: ", np.round(confusion_matrix(y_val, y_pred)[1][1]/(confusion_matrix(y_val, y_pred)[1][1]+confusion_matrix(y_val, y_pred)[0][1]), 2))
    print()

    # classification report
    print(classification_report(y_val, y_pred))

    # confusion matrix con seaborn
    cm = confusion_matrix(y_val,y_pred)
    sns.heatmap(cm, annot=True, fmt="d")
    # axes
    plt.xlabel("Predicted")
    # title
    plt.title("Confusion Matrix")
    plt.ylabel("True")
    plt.show()
    
    # plot roc curve
    try:
        y_pred_proba = grid.predict_proba(X_val)[::,1]
        fpr, tpr, _ = roc_curve(y_val,  y_pred_proba)
        auc = roc_auc_score(y_val, y_pred_proba)
        print(auc)
        plt.plot(fpr,tpr,label="auc="+str(auc))
        # axes
        plt.ylabel("True Positive Rate")
        plt.xlabel("False Positive Rate")
        # linea diagonal en gris claro
        plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
        # titulo
        plt.title("ROC Curve")
        plt.legend(loc=4)
        plt.show()
        precision, recall, _ = precision_recall_curve(y_val, y_pred_proba)

        # Plot the precision-recall curve
        plt.plot(recall, precision, marker='.')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.grid(True)
        plt.show()

        # predict probabilit
        # reliability diagram
        fop, mpv = calibration_curve(y_val, y_pred_proba, n_bins=10, normalize=True)
        # plot perfectly calibrated
        plt.plot([0, 1], [0, 1], linestyle='--')
        # plot model reliability
        plt.plot(mpv, fop, marker='.')
        plt.xlabel('Mean predicted value')
        plt.ylabel('Fraction of positives')
        plt.title('Calibration Curve')
        plt.grid(True)
        plt.show()

        DF = DF.append({"Modelo": str(model), "F1": f1_score(y_val, y_pred), "Precision": precision_score(y_val, y_pred), "Recall": recall_score(y_val, y_pred), "Balanced accuracy": balanced_accuracy_score(y_val, y_pred), "NPV": np.round(confusion_matrix(y_val, y_pred)[0][0]/(confusion_matrix(y_val, y_pred)[0][0]+confusion_matrix(y_val, y_pred)[1][0]), 2), "PPV": np.round(confusion_matrix(y_val, y_pred)[1][1]/(confusion_matrix(y_val, y_pred)[1][1]+confusion_matrix(y_val, y_pred)[0][1]), 2), "Time": execution_time, "AUC": roc_auc_score(y_val, y_pred_proba),"FPR":fpr,"TPR":tpr,"FOP":fop,"MPV":mpv}, ignore_index=True)
    except:
        fpr, tpr, _ = roc_curve(y_val,  y_pred)
        fop, mpv = calibration_curve(y_val, y_pred, n_bins=10, normalize=True)
        precision, recall, _ = precision_recall_curve(y_val, y_pred)
        DF = DF.append({"Modelo": str(model), "F1": f1_score(y_val, y_pred), "Precision": precision_score(y_val, y_pred), "Recall": recall_score(y_val, y_pred), "Balanced accuracy": balanced_accuracy_score(y_val, y_pred), "NPV": np.round(confusion_matrix(y_val, y_pred)[0][0]/(confusion_matrix(y_val, y_pred)[0][0]+confusion_matrix(y_val, y_pred)[1][0]), 2), "PPV": np.round(confusion_matrix(y_val, y_pred)[1][1]/(confusion_matrix(y_val, y_pred)[1][1]+confusion_matrix(y_val, y_pred)[0][1]), 2), "Time": execution_time, "AUC": roc_auc_score(y_val, y_pred),"FPR":fpr,"TPR":tpr,"FOP":fop,"MPV":mpv}, ignore_index=True)

    return DF

In [ ]:
SAME = pd.DataFrame(columns=["Modelo", "F1", "Precision", "Recall", "Balanced accuracy", "NPV", "PPV", "Time", "AUC","FPR","TPR","FOP","MPV","Cost"])

In [ ]:
TODAS2 = pd.DataFrame(columns=["Modelo", "F1", "Precision", "Recall", "Balanced accuracy", "NPV", "PPV", "Time", "AUC", "FPR", "TPR", "PR", "RE","Cost"])
SIGNIFICATIVAS2 = pd.DataFrame(columns=["Modelo", "F1", "Precision", "Recall", "Balanced accuracy", "NPV", "PPV", "Time", "AUC", "FPR", "TPR", "PR", "RE","Cost"])

TODAS3 = pd.DataFrame(columns=["Modelo", "F1", "Precision", "Recall", "Balanced accuracy", "NPV", "PPV", "Time", "AUC", "FPR", "TPR", "PR", "RE","Cost"])
SIGNIFICATIVAS3 = pd.DataFrame(columns=["Modelo", "F1", "Precision", "Recall", "Balanced accuracy", "NPV", "PPV", "Time", "AUC", "FPR", "TPR", "PR", "RE","Cost"])

TODAS4 = pd.DataFrame(columns=["Modelo", "F1", "Precision", "Recall", "Balanced accuracy", "NPV", "PPV", "Time", "AUC", "FPR", "TPR", "PR", "RE","Cost"])
SIGNIFICATIVAS4 = pd.DataFrame(columns=["Modelo", "F1", "Precision", "Recall", "Balanced accuracy", "NPV", "PPV", "Time", "AUC", "FPR", "TPR", "PR", "RE","Cost"])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
# random forest
from sklearn.ensemble import RandomForestClassifier
# xgboost
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
#import gridsearchcv
from sklearn.model_selection import GridSearchCV
# classification report
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
# XGBoost
from xgboost import XGBClassifier
# ada boost
from sklearn.ensemble import AdaBoostClassifier
# random forest
from sklearn.ensemble import RandomForestClassifier
# knn
from sklearn.neighbors import KNeighborsClassifier
# GAN network
from sklearn.ensemble import VotingClassifier

# no mostrar warnings
import warnings
warnings.filterwarnings('ignore')
models2 = [AdaBoostClassifier(),LogisticRegression(),SVC(probability=True),MLPClassifier(),DecisionTreeClassifier(),KNeighborsClassifier()]
models2_names = ["AdaBoostClassifier","LogisticRegression","SVC","MLPClassifier","DecisionTreeClassifier","KNeighborsClassifier"]
params2 = [{"n_estimators": [100, 200, 300, 400, 500], "learning_rate": [0.01, 0.1, 1, 10, 100], "algorithm": ["SAMME", "SAMME.R"]},
           {},
                {"C": [0.1, 1, 10, 100, 1000], "kernel": ["linear", "poly", "rbf", "sigmoid"], "gamma": ["scale", "auto"]},
                    {"hidden_layer_sizes": [(10,10,10), (100,100,100), (100,100), (3,5), (5,3)], "activation": ["identity", "logistic", "tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "learning_rate": ["constant", "invscaling", "adaptive"]},
                        {"criterion": ["gini", "entropy"], "max_depth": [None, 2, 3, 4, 5, 6, 7, 8, 9, 10], "min_samples_split": [2, 3, 4, 5, 6, 7, 8, 9, 10]},
                            {"n_neighbors": [3, 5, 7, 9, 11, 13, 15, 17, 19], "weights": ["uniform", "distance"], "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]}]

# entrenar los modelos con el dataset de entrenamiento y validarlos con el dataset de validacion
for i in range(len(models2)):
    warnings.filterwarnings('ignore')
    print("Model: ", models2_names[i])
    TODAS2 = train_model_gridsearch2(models2[i], X_train_todos, X_test_todos, y_train["TRT_60"], y_test["TRT_60"], params2[i],TODAS2)
    SIGNIFICATIVAS2 = train_model_gridsearch2(models2[i], X_train_final, X_test_final, y_train["TRT_60"], y_test["TRT_60"], params2[i],SIGNIFICATIVAS2)
    TODAS3 = train_model_gridsearch2(models2[i], X_train_todos65, X_test_todos, y_train_bi65t, y_test["TRT_65"], params2[i],TODAS3)
    SIGNIFICATIVAS3 =train_model_gridsearch2(models2[i], X_train_final65, X_test_final, y_train_bi65, y_test["TRT_65"], params2[i],SIGNIFICATIVAS3)
    TODAS4 = train_model_gridsearch2(models2[i], X_train_todos70, X_test_todos, y_train_bi70t, y_test["TRT_70"], params2[i],TODAS4)
    SIGNIFICATIVAS4 =train_model_gridsearch2(models2[i], X_train_final70, X_test_final, y_train_bi70, y_test["TRT_70"], params2[i],SIGNIFICATIVAS4)
    print("")

In [ ]:
SIGNIFICATIVAS2

In [ ]:
X_train_todos.columns

In [ ]:
# voting de svc, knn y mlp
models2 = [("KNN", KNeighborsClassifier()), ("LR", LogisticRegression()), ("MLP", MLPClassifier())]
voting = VotingClassifier(estimators=models2)
params2 = {"voting": ["hard", "soft"],"weights": [[1,1,1],[1,2,1],[1,1,2],[1,2,2],[2,1,1],[2,2,1],[2,1,2],[2,2,2]]}
print("Model: ", voting)
TODAS2 = train_model_gridsearch2(voting, X_train_todos, X_test_todos, y_train["TRT_60"], y_test["TRT_60"], params2,TODAS2)
SIGNIFICATIVAS2 = train_model_gridsearch2(voting, X_train_final, X_test_final, y_train["TRT_60"], y_test["TRT_60"], params2,SIGNIFICATIVAS2)
TODAS3 = train_model_gridsearch2(voting, X_train_todos65, X_test_todos, y_train_bi65t, y_test["TRT_65"], params2,TODAS3)
SIGNIFICATIVAS3 =train_model_gridsearch2(voting, X_train_final65, X_test_final, y_train_bi65, y_test["TRT_65"], params2,SIGNIFICATIVAS3)
TODAS4 = train_model_gridsearch2(voting, X_train_todos70, X_test_todos, y_train_bi70t, y_test["TRT_70"], params2,TODAS4)
SIGNIFICATIVAS4 =train_model_gridsearch2(voting, X_train_final70, X_test_final, y_train_bi70, y_test["TRT_70"], params2,SIGNIFICATIVAS4)
print("")

In [ ]:
SIGNIFICATIVAS4

In [ ]:
# quitar el 6 de todos los dataframes

TODAS2 = TODAS2.drop([6])
SIGNIFICATIVAS2 = SIGNIFICATIVAS2.drop([6])
TODAS3 = TODAS3.drop([6])
SIGNIFICATIVAS3 = SIGNIFICATIVAS3.drop([6])
TODAS4 = TODAS4.drop([6])
SIGNIFICATIVAS4 = SIGNIFICATIVAS4.drop([6])

In [ ]:
TODAS2

In [ ]:
SIGNIFICATIVAS2

In [ ]:
TODAS3

In [ ]:
SIGNIFICATIVAS3

In [ ]:
TODAS4

In [ ]:
SIGNIFICATIVAS4

In [ ]:
# guardar los resultados en un csv
TODAS2.to_csv("TODAS2.csv", index=False)
SIGNIFICATIVAS2.to_csv("SIGNIFICATIVAS2.csv", index=False)
TODAS3.to_csv("TODAS3.csv", index=False)
SIGNIFICATIVAS3.to_csv("SIGNIFICATIVAS3.csv", index=False)
TODAS4.to_csv("TODAS4.csv", index=False)
SIGNIFICATIVAS4.to_csv("SIGNIFICATIVAS4.csv", index=False)

In [ ]:
TODAS2.mean()

In [ ]:
SIGNIFICATIVAS2.mean()

In [ ]:
TODAS3.mean()

In [ ]:
SIGNIFICATIVAS3.mean()


In [ ]:
TODAS4.mean()

In [ ]:
SIGNIFICATIVAS4.mean()